### Lost in the Middle:长上下文精度问题

In [ ]:
! pip install sentence-transformers

In [1]:
from langchain.chains import LLMChain, StuffDocumentsChain
from langchain.document_transformers import LongContextReorder
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma

# 解决HuggingFace连接不上的问题
import os
os.environ['CURL_CA_BUNDLE'] = ''
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

embeddings = HuggingFaceBgeEmbeddings(model_name = "all-MiniLM-L6-v2")  # 使用HuggingFace托管的开源LLM做嵌入

text = [
    "篮球是一项伟大的运动。",
    "带我飞往月球是我最喜欢的歌曲之一。",
    "凯尔特人队是我最喜欢的球队。",
    "这是一篇关于波士顿凯尔特人的文件。",
    "我非常喜欢去看电影。",
    "波士顿凯尔特人队以20分的优势赢得了比赛。",
    "这只是一段随机的文字。",
    "《艾尔登法环》是过去15年最好的游戏之一。",
    "L.科内特是凯尔特人队最好的球员之一。",
    "拉里·伯德是一位标志性的NBA球员。"
]

retrieval = Chroma.from_texts(texts=text, embedding=embeddings).as_retriever(
    search_kwargs={"k": 10}
)

query = "关于凯尔特人队你知道什么"

# 根据相关性检索文档
docs = retrieval.get_relevant_documents(query)

docs


C:\Users\Administrator\AppData\Local\Temp\ipykernel_24864\3325544754.py:11: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(model_name = "all-MiniLM-L6-v2")  # 使用HuggingFace托管的开源LLM做嵌入
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24864\3325544754.py:33: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retrieval.get_relevant_documents(query)


[Document(metadata={}, page_content='凯尔特人队是我最喜欢的球队。'),
 Document(metadata={}, page_content='L.科内特是凯尔特人队最好的球员之一。'),
 Document(metadata={}, page_content='波士顿凯尔特人队以20分的优势赢得了比赛。'),
 Document(metadata={}, page_content='这是一篇关于波士顿凯尔特人的文件。'),
 Document(metadata={}, page_content='《艾尔登法环》是过去15年最好的游戏之一。'),
 Document(metadata={}, page_content='拉里·伯德是一位标志性的NBA球员。'),
 Document(metadata={}, page_content='我非常喜欢去看电影。'),
 Document(metadata={}, page_content='篮球是一项伟大的运动。'),
 Document(metadata={}, page_content='这只是一段随机的文字。'),
 Document(metadata={}, page_content='带我飞往月球是我最喜欢的歌曲之一。')]

In [2]:
# 对检索结果进行排序，根据论文的方案
# 问题相关性越低的内容块放在中间
# 问题相关性越高的内容块放在两端

reordering = LongContextReorder()

reorder_docs = reordering.transform_documents(docs)

# 头尾共有4个高相关性文档
reorder_docs

[Document(metadata={}, page_content='L.科内特是凯尔特人队最好的球员之一。'),
 Document(metadata={}, page_content='这是一篇关于波士顿凯尔特人的文件。'),
 Document(metadata={}, page_content='拉里·伯德是一位标志性的NBA球员。'),
 Document(metadata={}, page_content='篮球是一项伟大的运动。'),
 Document(metadata={}, page_content='带我飞往月球是我最喜欢的歌曲之一。'),
 Document(metadata={}, page_content='这只是一段随机的文字。'),
 Document(metadata={}, page_content='我非常喜欢去看电影。'),
 Document(metadata={}, page_content='《艾尔登法环》是过去15年最好的游戏之一。'),
 Document(metadata={}, page_content='波士顿凯尔特人队以20分的优势赢得了比赛。'),
 Document(metadata={}, page_content='凯尔特人队是我最喜欢的球队。')]

In [3]:
from langchain_community.llms import Tongyi
from langchain.prompts import PromptTemplate
import os

DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

llm = Tongyi(
    model="qwen-plus",
    api_key=DASHSCOPE_API_KEY,
    temperature=0,
    max_tokens=256,
)

document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}",
)

stuff_prompt_override = """
    Given this text extracts:
    ----------------------------------
    {context}
    ----------------------------------
    Please answer the question below. If you don't know the answer, just say "I don't know".
    {query}
"""

promt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context", "query"],
)

llm_chain = LLMChain(
    llm=llm,
    prompt=promt,
)

workchain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_prompt,
    document_variable_name="context",
)

# 运行工作流
workchain.run(
    input_documents=reorder_docs,
    query="凯尔特人是哪里的球队？",
)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_24864\3114908255.py:33: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24864\3114908255.py:38: LangChainDeprecationWarning: This class is deprecated. Use the `create_stuff_documents_chain` constructor instead. See migration guide here: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain/
  workchain = StuffDocumentsChain(
C:\Users\Administrator\AppData\Local\Temp\ipykernel_24864\3114908255.py:45: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  workchain.run(


'波士顿凯尔特人是来自美国马萨诸塞州波士顿的球队。'